In [1]:
import pandas as pd
import numpy as np
data_path = "July2021_for_test.xlsx"
df = pd.read_excel(data_path, usecols="A:Q", engine="openpyxl")
print(df["Плавок(окр)"].sum())

900


In [2]:
index = np.where (df["Плавок(окр)"] >=2)
for i in index[0]:
    count = df.iloc[i]["Плавок(окр)"]
    #print(count)
    #remain = df.at[i, "Плавок(окр)"]%18
    df.at[i, "Плавок(окр)"] = 1
    count -=1
    while(count>0):
        df1 = df.loc[[i],:]
        df1["Плавок(окр)"] = 1
        df = pd.concat([df, df1], ignore_index=True)
        count-=1
        
#    df1 = df.loc[[i],:]
#    df1["Плавок(окр)"] = remain
#    df = pd.concat([df, df1], ignore_index=True)

In [3]:
df["imax"] = df["Диапазон изменения d от 1"]*16

In [4]:
df = df.sort_values(by=["Диапазон изменения d от 1"]).reset_index()

In [5]:
df.loc[df["imax"] ==80, "imax"]=32
df.loc[df["imax"] ==240, "imax"]=80

In [6]:
df[df["imax"]>80]

,index,Номер(i),Толщ.,Шир.,Диапазон изменения d от 1,ПОРТ,АПК,Плавок(окр),Разрядов(L_i),У6,У7,У8,У9 Fix,У9,Подходит УНРС,Диапазон изменения k,Число переменных,Номер толщины(H_i),imax
18,771,106,250,2080,9,С-Петербург,1,1,3,0,0,1,0,1,2,"2,3",54,1,144
19,78,78,250,1850,9,Новороссийск,1,1,0,1,1,1,0,1,4,"0,1,2,3",0,1,144
20,102,102,250,2075,9,С-Петербург,1,1,3,0,0,1,0,1,2,"2,3",54,1,144
21,768,106,250,2080,9,С-Петербург,1,1,3,0,0,1,0,1,2,"2,3",54,1,144
22,747,102,250,2075,9,С-Петербург,1,1,3,0,0,1,0,1,2,"2,3",54,1,144
23,748,102,250,2075,9,С-Петербург,1,1,3,0,0,1,0,1,2,"2,3",54,1,144
24,749,102,250,2075,9,С-Петербург,1,1,3,0,0,1,0,1,2,"2,3",54,1,144
25,750,102,250,2075,9,С-Петербург,1,1,3,0,0,1,0,1,2,"2,3",54,1,144
26,769,106,250,2080,9,С-Петербург,1,1,3,0,0,1,0,1,2,"2,3",54,1,144
27,106,106,250,2080,9,С-Петербург,1,1,3,0,0,1,0,1,2,"2,3",54,1,144


In [7]:
# Array for thicknesses by number of heat
w_numbers = df["Номер толщины(H_i)"].to_numpy()
# Array for imax
j_imax = df["imax"].to_numpy() 

In [8]:
from amplify import (
    BinaryPoly,
    BinaryQuadraticModel,
    sum_poly,pair_sum,
    gen_symbols,
    Solver,
    decode_solution,
)
from amplify import Solver
from amplify.client import FixstarsClient
#from amplify.client.ocean import DWaveSamplerClient
from amplify.constraint import less_equal, equal_to, penalty

def flatten(t):
    return [item for sublist in t for item in sublist]

client = FixstarsClient()
client.token = "LvfkonaT7sLymsCqHyyQUp7JxHlo02CF"
client.parameters.timeout = 20000  # Timeout is 60 seconds

solver = Solver(client)

In [29]:
n = 80 #Number of timeslots for 5 days
#n = 40
m = 795 #Number of heats
#m = 360
w = 6 #Number of thicknesses
w_max = [3,4,0,1,1,1] #Max thicknesses per UNRS

x = gen_symbols(BinaryPoly,m,n) #Variable x - heat for timeslot


In [30]:
# Condition for min(n, imax[j])
for j in range (m):
    if (df.loc[j, "imax"]<n):
        for i in range (df.loc[j, "imax"], n):
            x[j][i]=0

In [ ]:
# 5.1. В одном слоте 4 плавки по кол-ву УНРС
#A = 1 #Coeff
#H1 = A * sum_poly(n, lambda i: (4-sum_poly(m, lambda j:(x[j][i])))**2)

In [ ]:
#H1.count()

In [31]:
# Либо для 5.1. Constraints on columns
col_constraints = [
    equal_to(sum_poly([x[j][i] for j in range(m)]), 4) for i in range(n)
]

In [32]:
H1_constr = sum(col_constraints)

In [33]:
# 5.2 Deadline
S = [int(df.loc[j,"imax"]<=n) for j in range(m)] #плавка должна быть разлита обязательно - взять из df
B=1
R = 1

In [34]:
H2_0 = sum_poly(m, lambda j: S[j]*(1 - sum_poly(n, lambda i:(x[j][i])))**2)

In [ ]:
#H2_1 = sum_poly(m, lambda j:(1 - S[j]*pair_sum(n, lambda i:(x[j][i]))))

In [35]:
# Либо для 5.2 H2_1 Constraints on rows
row_constraints = 0
for j in range(m):
    if (1-S[j]):
        row_constraints += less_equal(sum_poly([x[j][i] for i in range(n)]), 1)

H2_1_constr = sum(row_constraints)

In [36]:
H2 = B*H2_0 + R*H2_1_constr

In [37]:
H2.num_logical_vars

62736

In [40]:
# 5.3 Ограничение на количество плавок одной толщины в каждом ts
w_sum = [[0 for x in range(w)] for y in range(n)] #array for sums of each thickness for each ts
#constr_w = 0
# Number of heats for each thickness for each ts
# !! ошибочный прогон
# for i in range(n):
#    for w_ind in range(w):
#        label = "Thickness "+str(w_ind)+" for ts" + str(i)
#        #print(w_numbers[j])
#        if (w_numbers[j]==w_ind):
#            for j in range(m):
#                w_sum[i][w_ind]+= (int(w_numbers[j]==w_ind))*x[j][i]
#                #print("!")   
#        if w_sum[i][w_ind]!=0:
#            #print(w_sum[i][w_ind])
#            constr_w += less_equal(w_sum[i][w_ind], w_max[w_ind], label=label) 
for i in range(n):
    for w_ind in range(w):
        label = "Thickness "+str(w_ind)+" for ts" + str(i)
        for j in range(m):
            w_sum[i][w_ind]+= (int(w_numbers[j]==w_ind))*x[j][i]
        #constr_w += less_equal(w_sum[i][w_ind], w_max[w_ind], label=label)

In [ ]:
x_w0 = gen_symbols(BinaryPoly,n*m,(w_max[0]+1,n))
x_w1 = gen_symbols(BinaryPoly,n*m+len(x_w0)*n,(w_max[1]+1,n))
x_w2 = gen_symbols(BinaryPoly,n*m+(len(x_w0)+len(x_w1))*n,(w_max[2]+1,n))
x_w3 = gen_symbols(BinaryPoly,n*m+(len(x_w0)+len(x_w1)+len(x_w2))*n,(w_max[3]+1,n))
x_w4 = gen_symbols(BinaryPoly,n*m+(len(x_w0)+len(x_w1)+len(x_w2)+len(x_w3))*n,(w_max[4]+1,n))
x_w5 = gen_symbols(BinaryPoly,n*m+(len(x_w0)+len(x_w1)+len(x_w2)+len(x_w3)+len(x_w4))*n,(w_max[5]+1,n))

In [ ]:
def h3_w(w_num, i_ind, x_w):
    H3_i_w = 0
    H3_i_w += w_sum[i_ind][w_num]
    for ww in range (1, w_max[w_ind]+1):
        H3_i_w +=ww*x_w[ww][i_ind]
    H3_i_w = (H3_i_w - w_max[w_ind])**2 
    return(H3_i_w)
    
H3 = 0
for i in range (n):
    for w_ind in range(w):
        if (w_ind == 0): 
            H3+=h3_w(w_ind, i, x_w0)
        elif (w_ind == 1): 
            H3+=h3_w(w_ind, i, x_w1)
        elif (w_ind == 2): 
            H3+=h3_w(w_ind, i, x_w2)  
        elif (w_ind == 3): 
            H3+=h3_w(w_ind, i, x_w3)
        elif (w_ind == 4): 
            H3+=h3_w(w_ind, i, x_w4)
        elif (w_ind == 5): 
            H3+=h3_w(w_ind, i, x_w5)

In [41]:
#Create array x_w_full for slack variables
x_w_full = []
x_w_start = n*m
for w_ind in range(w):
    w_max_ind = w_max[w_ind]+1
    x_w_full.append(gen_symbols(BinaryPoly, x_w_start, (w_max_ind,n)))
    x_w_start += w_max_ind*n
        

In [42]:
H3 = 0
x_w_start = n*m
for i in range (n):
    for w_ind in range(w):
        H3_i_w = 0
        H3_i_w += w_sum[i][w_ind]
        for ww in range (1, w_max[w_ind]+1):
            x_w_start += ww
            H3_i_w +=ww*x_w_full[w_ind][ww][i]
        x_w_start += w_max[w_ind]*n    
        H3_i_w = (H3_i_w - w_max[w_ind])**2 
        H3+=H3_i_w   

In [43]:
H3.count()

11864945

In [ ]:
#H3 = sum(constr_w)

In [ ]:
# H4 APK(L) ACV(V)
K = 1 #Coeff
L = []
V = [] # TBD
H4 = 0
for j in range (m):
    L.append(df.loc[j,"АПК"])   
for i in range(n):
    H4 += less_equal(sum_poly(m, lambda j:(L[j]*x[j][i])), 2)
H4 = K*sum(H4)    

In [ ]:
# 5.5 Логистические расходы
F = 1 #Coeff
H5 = F*(sum_poly(m, lambda j: (j_imax[j] - sum_poly(n, lambda i:(i*x[j][i])))**2))

In [ ]:
# 5.6 Перенастройки по толщине
# Для каждой толщины считаем, сколько плавок данной толщины делалось в соседних ts. Минимизируем дельту
# We can use array w_sum = [[0 for x in range(w)] for y in range(n)]
Q = 1
delta_w = 0
w_delta = []
for i in range (n-1):
    for w_ind in range(w):
        label = "Delta thickness "+str(w_ind)+" for ts" + str(i)
        if (w_sum[i+1][w_ind]!=0 or w_sum[i+1][w_ind]!=0):
            w_delta.append(penalty(0.5*((w_sum[i+1][w_ind] - w_sum[i][w_ind])**2), label))     
H6 = Q*sum(w_delta)       

In [ ]:
len(H6)

In [ ]:
H = H5 + H2_0 + H1_constr + H2_1_constr + H3 + H4 + H6

In [44]:
model = H3 + H2_0 + H1_constr + H2_1_constr

In [45]:
model.num_logical_vars

63536

In [ ]:
result = solver.solve(model)
if len(result) == 0:
    raise RuntimeError("Any one of constraints is not satisfied.")

energy, values = result[0].energy, result[0].values

x_values = decode_solution(x, values)

print(energy)
    
x_values_array = np.asarray(x_values, dtype = np.int32)
np.savetxt("slabs-"+str(pd.Timestamp.now())+".csv", x_values_array, fmt='%i', delimiter=",")

In [ ]:
for i in range (n):
    print("Timeslot ", i)
    for j in range(m):
        if (x_values[j][i]):
            print("Heat ", j)
            print ("Thickness ", w_numbers[j])
            print (x_values[j][i])
    
    

In [26]:
client_result = solver.client_result
client_result.annealing_time_ms

19730

In [27]:
client_result.timing.cpu_time

2545.650934000001

In [ ]:
x_values_array = np.asarray(x_values, dtype = np.int32)
np.savetxt("slabs-"+str(pd.Timestamp.now())+".csv", x_values_array, fmt='%i', delimiter=",")

In [ ]:
def get_assignments(x_ij, rng=40):
    casts_assign = [[] for y in range(rng)]
    cast = 0
    for c in x_ij:
        if 1 in c:
            casts_assign[list(c).index(1)].append(cast)
        cast += 1
    return casts_assign

def get_thickness_by_cast_num(c):
    return df["Номер толщины(H_i)"][c]

#xij = np.loadtxt(open('/home/eugene/__Work/Slabs/Code/slabs-2021-11-09 00 01 03.398912.csv', "r"), delimiter=",")  # or just take the decoded solution 2D-array

casts_assign = get_assignments(x_values)
print(casts_assign)

thickness_assign = [list(map(get_thickness_by_cast_num, slot)) for slot in casts_assign]
print(thickness_assign)

In [ ]:
def check_thickness_assign(d):
    if d[0] > 3:
        print('Thickness 0 casts number violation')
    if d[1] > 4:
        print('Thickness 1 casts number violation')
    if d[2] > 0:
        print('Thickness 2 casts number violation')
    if d[3] > 1:
        print('Thickness 3 casts number violation')
    if d[4] > 1:
        print('Thickness 4 casts number violation')
    if d[5] > 1:
        print('Thickness 5 casts number violation')
    else:
        print("Ok")

slot_num = 0
for slot in thickness_assign:
    d = {0:0, 1:0, 2:0, 3:0, 4:0, 5:0}
    for thickness in slot:
        d[thickness] += 1
    print(str(d) + 'in slot ' + str(slot_num))
    check_thickness_assign(d)

    slot_num += 1

In [ ]:
len(x_values[0])